In [1]:
import os
import sys
import polars as pl
from typing import List, Dict, Callable
from dataclasses import dataclass
from main import DataDictionary
from variables import default_classifications

In [2]:
# Creates data dictionary
dictionary:DataDictionary = DataDictionary()
# Assigns default classifications
dictionary.apply_assign(default_classifications,dictionary.assign_classification)
# Prints the entries as a dataframe
dictionary.frame_dictionary()

Field,Description,Classification
str,str,str
"""active_account""","""account is active""","""target"""
"""delinquent_account""","""account is delinquent""","""target"""
"""ident_monitor_opt""","""opted into identity monitoring""","""actionable"""
"""Num_Bk_Accts""","""number of banking accounts""","""actionable"""
"""income""","""annual income""","""actionable"""
…,…,…
"""cc_inq_72hr""","""number of credit card inquirie…","""actionable"""
"""cc_inq_10dy""","""number of credit card inquirie…","""actionable"""
"""pl_inq_72hr""","""number of personal loan inquir…","""actionable"""
